In [69]:
import re
from nltk.corpus import stopwords
import pandas as pd
# 1x in je notebook draaien (als je dat nog niet had gedaan)
import nltk
nltk.download('stopwords')

df = pd.read_csv("Uitgebreide_VKM_dataset_cleaned2.csv")

# Algemene Nederlandse stopwoorden
DUTCH_STOPWORDS = set(stopwords.words("dutch"))

# Extra ruiswoorden die je NIET als match wilt zien

TEXT_STOPWORDS = DUTCH_STOPWORDS

def clean_text_for_matching(text: str) -> str:
    """Maak tekst schoon voor matching / uitleg."""
    text = str(text).lower()
    # haal alleen 'normale' woorden eruit
    tokens = re.findall(r"[0-9a-zA-ZÀ-ÖØ-öø-ÿ]+", text)
    # filter stopwoorden en superkorte zooi
    tokens = [
        t for t in tokens
        if t not in TEXT_STOPWORDS and len(t) > 2
    ]
    return " ".join(tokens)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jeroe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [55]:
# Als je al combined_text hebt, gebruik die.
if "combined_text" in df.columns:
    base_text = df["combined_text"]
else:
    # Fallback: zelf combineren (pas aan naar wat jij wilt gebruiken)
    df["combined_text"] = (
        df["name"].fillna("") + " " +
        df["shortdescription"].fillna("") + " " +
        df["description"].fillna("") + " " +
        df["content"].fillna("") + " " +
        df["module_tags"].astype(str).fillna("") +" " +
        df["location"].astype(str).fillna("")
    )
    base_text = df["combined_text"]

# Schoongemaakte tekst voor recommender + uitleg
df["clean_text"] = base_text.apply(clean_text_for_matching)


In [56]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Vectorizer trainen op de opgeschoonde tekst
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["clean_text"])

# Handig voor later bij match_terms
FEATURE_NAMES = vectorizer.get_feature_names_out()


In [57]:
import numpy as np

def extract_match_terms(student_vec, module_vec, max_terms: int = 8):
    """
    Zoek de overlappende woorden tussen studentprofiel en module.
    Geeft een lijst met duidelijke keywords terug.
    """
    student_idx = set(student_vec.nonzero()[1])
    module_idx = set(module_vec.nonzero()[1])
    shared_idx = list(student_idx & module_idx)

    # Sorteer op naam (kan ook op frequentie, maar dit is prima voor nu)
    shared_idx = sorted(shared_idx)

    terms = [FEATURE_NAMES[i] for i in shared_idx]
    return terms[:max_terms]


In [96]:
def build_reason(match_terms):
    if not match_terms:
        return "Deze module sluit aan bij je interesses op basis van de tekstuele overeenkomsten."
    if len(match_terms) == 1:
        return f"Deze module sluit aan bij je interesse in '{match_terms[0]}'."
    if len(match_terms) == 2:
        return f"Deze module sluit aan bij je interesses in '{match_terms[0]}' en '{match_terms[1]}'."
    # 3 of meer
    hoofd = ", ".join(match_terms[:-1])
    laatste = match_terms[-1]
    return f"Deze module sluit aan bij je interesses in {hoofd} en {laatste}."

def recommend_modules(student_profile: str, top_n: int = 5, studycredit: int = None, level: str = None, location: str = None):
    
    filtered_df = df.copy()
    if studycredit is not None:
        filtered_df = filtered_df[filtered_df["studycredit"] == studycredit]

    if level is not None:
        filtered_df = filtered_df[filtered_df["level"] == level]
    
    if location is not None:
        filtered_df = filtered_df[filtered_df["location"].str.contains(location, na=False)]
    

    """
    Geeft top N modules voor een studentprofiel,
    inclusief similarity, match_terms en een NL-uitleg.
    """
    # studenttekst schoonmaken
    clean_profile = clean_text_for_matching(student_profile)
    student_vec = vectorizer.transform([clean_profile])

    filtered_matrix = X[filtered_df.index]

    # cosine similarity
    sims = cosine_similarity(student_vec, filtered_matrix).flatten()
    df_result = filtered_df.copy()
    df_result["similarity"] = sims

    # Top N pakken
    top = df_result.sort_values("similarity", ascending=False).head(top_n)

    # match_terms + reden per rij
    match_terms_list = []
    reasons = []

    for idx in top.index:
        module_vec = X[idx]
        terms = extract_match_terms(student_vec, module_vec)
        match_terms_list.append(terms)
        reasons.append(build_reason(terms))

    top = top[["id","name", "shortdescription", "module_tags", "similarity", "location", "studycredit"]].copy()
    top["match_terms"] = match_terms_list
    top["reason"] = reasons

    return top


In [ ]:
sam_profile = """
Ik ben derdejaarsstudent en ik heb veel interesse in zorginnovatie,
data en AI. Ik wil graag modules volgen over zorgtechnologie, 
innovatie in de zorg en creatieve manieren om met technologie impact te maken.
"""

pd.set_option("display.max_colwidth", None)

aanbevelingen = recommend_modules(sam_profile, top_n=5, studycredit=30, location="Breda")
display(aanbevelingen)

NameError: name 'pd' is not defined